In [1]:
#import magic and libraries
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [8]:
#import data set and clean up
charity = pd.read_csv('Charity.csv')
dummies = pd.get_dummies(charity.RenewalsOrAcq)
charity = pd.concat([charity, dummies], axis=1)
charity.drop(['Renewals'], inplace=True, axis=1)
charity.drop(['dob'], inplace=True, axis=1)
charity.drop(['RenewalsOrAcq'], inplace=True, axis=1)
charity.drop(['suburb'], inplace=True, axis=1)
charity.drop(['state'], inplace=True, axis=1)
charity = charity.dropna(subset=['age'])
charity['age'] = charity['age'].convert_objects(convert_numeric=True)
charity = charity.dropna(subset=['age'])
charity['postcode'] = charity['postcode'].astype(str)
charity.head()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,postcode,num_streets,collection_amount,donation_amount,total_received,age,weekends,profitable,Acq
0,801,2,315.00,0,315.00,72,4,1,0
1,801,6,303.55,0,303.55,65,4,1,1
2,804,4,250.00,0,250.00,80,4,1,0
3,804,2,28.60,50,78.60,57,4,1,0
5,810,2,521.50,0,521.50,67,4,1,1


In [9]:
#have a look at the data
charity.describe(include='all')

,postcode,num_streets,collection_amount,donation_amount,total_received,age,weekends,profitable,Acq
count,26597,26597.000000,26597.000000,26597.000000,26597.000000,26597.000000,26597.000000,26597.000000,26597.000000
unique,1772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,4350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.881716,77.967018,3.738053,81.705071,62.471858,3.352859,0.414107,0.613979
std,NaN,2.548462,71.066831,18.735377,70.878127,14.066151,0.870711,0.492576,0.486845
min,NaN,0.000000,-414.950000,-150.000000,-500.000000,0.000000,2.000000,0.000000,0.000000
25%,NaN,1.000000,37.000000,0.000000,40.000000,54.000000,2.000000,0.000000,0.000000
50%,NaN,2.000000,61.000000,0.000000,64.000000,65.000000,4.000000,0.000000,1.000000
75%,NaN,3.000000,100.000000,0.000000,101.300000,72.000000,4.000000,1.000000,1.000000


In [10]:
#look for any correlations
charity.corr()

,num_streets,collection_amount,donation_amount,total_received,age,weekends,profitable,Acq
num_streets,1.000000,-0.020782,-0.011124,-0.023778,-0.005437,0.020047,-0.034242,0.058324
collection_amount,-0.020782,1.000000,-0.141874,0.965161,0.176009,0.091123,0.631454,-0.174396
donation_amount,-0.011124,-0.141874,1.000000,0.122081,0.041046,0.046338,0.061930,0.032654
total_received,-0.023778,0.965161,0.122081,1.000000,0.187327,0.103615,0.649506,-0.166229
age,-0.005437,0.176009,0.041046,0.187327,1.000000,0.124830,0.186369,-0.117583
weekends,0.020047,0.091123,0.046338,0.103615,0.124830,1.000000,0.107270,-0.054035
profitable,-0.034242,0.631454,0.061930,0.649506,0.186369,0.107270,1.000000,-0.153555
Acq,0.058324,-0.174396,0.032654,-0.166229,-0.117583,-0.054035,-0.153555,1.000000


In [14]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression

feature_cols = ['age', 'num_streets', 'weekends', 'Acq', 'postcode']
X = charity[feature_cols]
y = charity.total_received
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred_class = linreg.predict(X_test)
from sklearn import metrics
print metrics.r2_score(y_test, y_pred_class)


0.0637056411947


In [15]:
#add in ATO data and see if I can improve my model
tax = pd.read_csv('2014tax2.csv')
tax['postcode'] = tax['postcode'].astype(str)
charity_new = pd.merge(charity, tax, on='postcode')
charity_new.head()
print charity_new.columns



Index([u'postcode', u'num_streets', u'collection_amount', u'donation_amount',
       u'total_received', u'age', u'weekends', u'profitable', u'Acq',
       u'tax individuals', u'avg tax income', u'avg gross tax paid',
       u'avg net tax paid', u'avg salary', u'no pensions', u'avg pension',
       u'avg income', u'no donations', u'value donations',
       u'avg donation value'],
      dtype='object')


In [16]:
feature_cols = ['age', 'Acq', 'postcode', 'avg donation value', 'no donations', 'avg tax income']
X = charity_new[feature_cols]
y = charity_new.total_received
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred_class = linreg.predict(X_test)
from sklearn import metrics
print metrics.r2_score(y_test, y_pred_class)

0.0993599548147


In [21]:
#X = charity_new.filter(regex="[^total_received|^collection_amount|^donation_amount]")
X = charity_new.filter(regex="[^total_received|^collection_amount|^donation_amount|^profitable]")
y = charity_new.total_received
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred_class = linreg.predict(X_test)
from sklearn import metrics
print metrics.r2_score(y_test, y_pred_class)

0.102058468842


In [23]:
from sklearn import linear_model

# The coefficients
print('Coefficients: \n', linreg.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((linreg.predict(X_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % linreg.score(X_test, y_test))

('Coefficients: \n', array([ -4.70462402e-03,   3.66004264e-01,   8.08894986e-01,
         3.95121523e+00,  -2.12703638e+01,   9.42851949e-04,
         9.03273620e-03,  -5.56748393e-03,  -4.49999525e-03,
        -5.86257208e-04,  -1.41268416e-02,  -2.29381599e-04,
        -3.97364360e-03,  -2.00821215e-03,   1.09501961e-06,
        -5.75516007e-04]))
Residual sum of squares: 4068.82
Variance score: 0.10


In [24]:
# evaluate the model using 10-fold cross-validation
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(LinearRegression(), X, y, scoring='r2', cv=100)
print scores
print scores.mean()

[ 0.01488777  0.01567767  0.02370409  0.06592724  0.04152667  0.09522366
  0.03928696  0.05118205  0.06125715  0.06946681  0.04870453  0.063594
  0.060155   -0.38258146  0.08738936  0.07881185  0.04207142  0.06293442
  0.01173662  0.09966596  0.04069153  0.04586949  0.04856146  0.06955606
  0.09917994  0.02666496  0.06560108  0.04776459 -0.00835429  0.1329974
  0.12200442 -0.01378303  0.07843217 -0.01285094 -0.00406372  0.03225533
  0.05358413  0.05159167  0.06228723  0.12787676  0.05951514  0.09963621
  0.05427837  0.06788288  0.09094315  0.04264327  0.07814995  0.07158381
  0.08459726  0.00909945  0.01228254  0.00768033  0.08893874  0.04138621
  0.05732362  0.0768205   0.05916222 -0.01488988  0.03213476 -0.08843557
 -0.04210335  0.07744621  0.00435551  0.06957039 -0.029769   -0.0398822
 -0.05574992 -0.06484123  0.03412308 -0.06857012  0.03544881  0.06218148
  0.0646901   0.01787141 -0.00678724  0.00634505 -0.02119229  0.01127672
  0.00304973  0.07961486  0.04399712  0.07001123  0.068

In [ ]:
#this is really bad - linear regression not really doing it for me..

In [25]:
from sklearn.ensemble import RandomForestClassifier
from numpy import genfromtxt, savetxt

def main():
    #create the training & test sets, skipping the header row with [1:]
    dataset = charity_new[1:]    
    target = y_train
    train = X_train
    test = X_test
    
    #create and train the random forest
    #multi-core CPUs can use: rf = RandomForestClassifier(n_estimators=100, n_jobs=2)
    rf = RandomForestClassifier(n_estimators=100)
    rf.fit(train, target)

    savetxt('Data/submission2.csv', rf.predict(test), delimiter=',', fmt='%f')

if __name__=="__main__":
    main()

ValueError: Unknown label type: array([[ 49. ],
       [ 84. ],
       [ 72.2],
       ..., 
       [ 30. ],
       [ 41. ],
       [ 94. ]])